In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
sys.path.append(os.path.abspath(os.path.join('../../..')))
from apikeys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Portland,US,77.63,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
1,10,Koroni,GR,82.08,clear sky,36.8000,21.9500,Apartments Xenios Zeus 1
2,17,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,21,Kahului,US,87.82,few clouds,20.8947,-156.4700,Maui Seaside Hotel
4,23,Alofi,NU,78.69,overcast clouds,-19.0595,-169.9187,Taloa Heights
...,...,...,...,...,...,...,...,...
204,651,Gat,IL,79.48,clear sky,31.6100,34.7642,OlusHome
205,656,Awbari,LY,82.08,clear sky,26.5921,12.7805,فندق أوباري
206,657,Marsh Harbour,BS,83.86,clear sky,26.5412,-77.0636,Abaco Beach Resort
207,660,Cabangahan,PH,78.58,light rain,9.0781,122.9469,Centralle District Beach Resort


In [3]:
# Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# Add a marker layer for each city to the map.
fig = gmaps.figure(center=(40.0, 30.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method.

vacation_start = vacation_df.loc[(vacation_df["City"] == "Portland")]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "North Bend")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Eureka")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Pacific Grove")]

In [6]:
# Get the latitude-longitude pairs as tuples from each city DataFrame
start = list(vacation_start[['Lat', 'Lng']].itertuples(index=False, name=None))[0]
end = list(vacation_end[['Lat', 'Lng']].itertuples(index=False, name=None))[0]
stop1 = list(vacation_stop1[['Lat', 'Lng']].itertuples(index=False, name=None))[0]
stop2 = list(vacation_stop2[['Lat', 'Lng']].itertuples(index=False, name=None))[0]
stop3 = list(vacation_stop3[['Lat', 'Lng']].itertuples(index=False, name=None))[0]
destinations = [start, stop1, stop2, stop3]

In [17]:
# Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints.
fig2 = gmaps.figure()
dir_layer = gmaps.directions_layer(start, end, waypoints=destinations[1:])
fig2.add_layer(dir_layer)
fig2

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Portland,US,77.63,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
1,63,North Bend,US,77.23,scattered clouds,43.4065,-124.2243,Red Lion Hotel Coos Bay
2,288,Eureka,US,78.67,few clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
3,456,Pacific Grove,US,86.38,few clouds,36.6177,-121.9166,Lovers Point Inn


In [9]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

In [16]:
# Add a marker layer for each city to the map.
# Disable markers from previous layer, so new markers may be placed
dir_layer.show_markers = False

# Set and add marker layer
marker_layer = gmaps.marker_layer(destinations, info_box_content=hotel_info)
fig2.add_layer(marker_layer)

fig2

Figure(layout=FigureLayout(height='420px'))